# Chudnovsky Algorithm

- toc: true
- branch: master
- badges: true
- comments: true
- author: Kim Jeewoo
- categories: [Mathematics, Python]
- image: images/pi_logo.jpg

# Chudnovsky Algorithm

The Chudnovsky algorithm is a fast method for calculating the digits of π.

# Reference

https://en.wikipedia.org/wiki/Chudnovsky_algorithm

# Code

In [1]:
import math
from gmpy2 import mpz
from time import time
import gmpy2

def pi_chudnovsky_bs(digits):
    """
    Compute int(pi * 10**digits)

    This is done using Chudnovsky's series with binary splitting
    """
    C = 640320
    C3_OVER_24 = C**3 // 24
    def bs(a, b):
        """
        Computes the terms for binary splitting the Chudnovsky infinite series

        a(a) = +/- (13591409 + 545140134*a)
        p(a) = (6*a-5)*(2*a-1)*(6*a-1)
        b(a) = 1
        q(a) = a*a*a*C3_OVER_24

        returns P(a,b), Q(a,b) and T(a,b)
        """
        if b - a == 1:
            # Directly compute P(a,a+1), Q(a,a+1) and T(a,a+1)
            if a == 0:
                Pab = Qab = mpz(1)
            else:
                Pab = mpz((6*a-5)*(2*a-1)*(6*a-1))
                Qab = mpz(a*a*a*C3_OVER_24)
            Tab = Pab * (13591409 + 545140134*a) # a(a) * p(a)
            if a & 1:
                Tab = -Tab
        else:
            # Recursively compute P(a,b), Q(a,b) and T(a,b)
            # m is the midpoint of a and b
            m = (a + b) // 2
            # Recursively calculate P(a,m), Q(a,m) and T(a,m)
            Pam, Qam, Tam = bs(a, m)
            # Recursively calculate P(m,b), Q(m,b) and T(m,b)
            Pmb, Qmb, Tmb = bs(m, b)
            # Now combine
            Pab = Pam * Pmb
            Qab = Qam * Qmb
            Tab = Qmb * Tam + Pam * Tmb
        return Pab, Qab, Tab
    # how many terms to compute
    DIGITS_PER_TERM = math.log10(C3_OVER_24/6/2/6)
    N = int(digits/DIGITS_PER_TERM + 1)
    # Calclate P(0,N) and Q(0,N)
    P, Q, T = bs(0, N)
    one_squared = mpz(10)**(2*digits)
    sqrtC = gmpy2.isqrt(10005*one_squared)
    return (Q*426880*sqrtC) // T

# The last 5 digits or pi for various numbers of digits
check_digits = {
        100 : 70679,
       1000 :  1989,
      10000 : 75678,
     100000 : 24646,
    1000000 : 58151,
   10000000 : 55897,
}

if __name__ == "__main__":
    #raise SystemExit
    for log10_digits in range(1,8): # 10자리 ~ 1000만자리
        digits = 10**log10_digits
        start =time()
        pi = pi_chudnovsky_bs(digits)
        f = open('d_' + str(digits), 'w')
        f.write(str(pi))
        f.close()
        print("chudnovsky_gmpy_mpz_bs: digits",digits,"time",time()-start)
        if digits in check_digits:
            last_five_digits = pi % 100000
            if check_digits[digits] == last_five_digits:
                print("Last 5 digits %05d OK" % last_five_digits)
            else:
                print("Last 5 digits %05d wrong should be %05d" % (last_five_digits, check_digits[digits]))

chudnovsky_gmpy_mpz_bs: digits 10 time 0.002651214599609375
chudnovsky_gmpy_mpz_bs: digits 100 time 0.0008170604705810547
Last 5 digits 70679 OK
chudnovsky_gmpy_mpz_bs: digits 1000 time 0.0013988018035888672
Last 5 digits 01989 OK
chudnovsky_gmpy_mpz_bs: digits 10000 time 0.0020301342010498047
Last 5 digits 75678 OK
chudnovsky_gmpy_mpz_bs: digits 100000 time 0.03248786926269531
Last 5 digits 24646 OK
chudnovsky_gmpy_mpz_bs: digits 1000000 time 0.5917906761169434
Last 5 digits 58151 OK
chudnovsky_gmpy_mpz_bs: digits 10000000 time 11.120594263076782
Last 5 digits 55897 OK
